# Configuração do Ambiente e Criação do Schema Gold

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, lit, sequence, explode, to_date, year, quarter, month, 
    dayofmonth, dayofweek, dayofyear, date_format, when, expr, 
    countDistinct, sum, round, avg,
    coalesce 
)
from pyspark.sql.types import DecimalType, DateType
from pyspark.sql.window import Window

# Variáveis de ambiente
CATALOG_NAME = "`workspace`" 
SCHEMA_NAME = "olist_ecommerce"

# Configuração da sessão no Unity Catalog
spark.sql(f"USE CATALOG {CATALOG_NAME}")
spark.sql(f"USE SCHEMA {SCHEMA_NAME}")

# Criação do schema GOLD
spark.sql("CREATE SCHEMA IF NOT EXISTS gold COMMENT 'Dados consolidados, agregados e prontos para BI'")

print(f"✅ Configuração OK. Lendo de {SCHEMA_NAME}.silver e escrevendo em {SCHEMA_NAME}.gold.")

✅ Configuração OK. Lendo de olist_ecommerce.silver e escrevendo em olist_ecommerce.gold.


# 1º Projeto: Área de Logística (Vendas por Localidade)

#### Criação da Tabela gold.ft_vendas_consumidor_local

In [0]:
df_total = spark.read.table("silver.pedido_total")
df_consumidores = spark.read.table("silver.ft_consumidores")

# Join e Seleção
df_gold_local = (df_total
    .join(df_consumidores, ["id_consumidor"], "inner")
    .select(
        col("id_pedido").cast("STRING"),
        col("id_consumidor").cast("STRING"),
        col("valor_total_pago_brl").cast(DecimalType(12, 2)).alias("valor_total_pedido_brl"),
        col("cidade").cast("STRING"),
        col("estado").cast("STRING"),
        to_date(col("data_pedido"), "yyyy-MM-dd").alias("data_pedido").cast(DateType()),
    )
    .dropDuplicates(["id_pedido"]) 
)

# Salvar na Camada Gold (Requisito: modo overwrite)
target_table = "gold.ft_vendas_consumidor_local"
(df_gold_local.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable(target_table))

print(f"✅ Fato Gold '{target_table}' criada com sucesso.")

✅ Fato Gold 'gold.ft_vendas_consumidor_local' criada com sucesso.


#### Criação da View gold.view_total_compras_por_consumidor

In [0]:
spark.sql("""
CREATE OR REPLACE VIEW gold.view_total_compras_por_consumidor
AS
SELECT
    cidade,
    estado,
    COUNT(id_pedido) AS quantidade_vendas,     
    SUM(valor_total_pedido_brl) AS valor_total_localidade
FROM gold.ft_vendas_consumidor_local
GROUP BY cidade, estado
ORDER BY valor_total_localidade DESC
""")

print("✅ View 'gold.view_total_compras_por_consumidor' criada com sucesso.")

✅ View 'gold.view_total_compras_por_consumidor' criada com sucesso.


#### Consulta de Negócio

In [0]:
print("📈 Consulta 1.3: Total de Vendas por Estado")
spark.sql("""
SELECT
    estado,
    ROUND(SUM(valor_total_localidade), 2) AS receita_total_brl
FROM gold.view_total_compras_por_consumidor
GROUP BY estado
ORDER BY receita_total_brl DESC
""").show(5)

📈 Consulta 1.3: Total de Vendas por Estado
+------+-----------------+
|estado|receita_total_brl|
+------+-----------------+
|    SP|       5998226.96|
|    RJ|       2144379.69|
|    MG|       1872257.26|
|    RS|        890898.54|
|    PR|        811156.38|
+------+-----------------+
only showing top 5 rows


# 2º Projeto: Área de Logística (Análise de Atrasos)

#### Criação da Tabela gold.ft_atrasos_pedidos_local_vendedor

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

df_pedidos = spark.read.table("silver.ft_pedidos")
df_consumidores = spark.read.table("silver.ft_consumidores")
df_itens = spark.read.table("silver.ft_itens_pedidos")

# Agrega itens para obter o id_vendedor
df_vendedor_por_pedido = df_itens.select("id_pedido", "id_vendedor").dropDuplicates()

# Join e Seleção
df_gold_atrasos = (df_pedidos
    .join(df_consumidores, ["id_consumidor"], "inner") 
    .join(df_vendedor_por_pedido, ["id_pedido"], "inner")
    .select(
        col("id_pedido").cast("STRING"),
        col("id_vendedor").cast("STRING"),
        col("id_consumidor").cast("STRING"),
        col("entrega_no_prazo").cast("STRING"),
        col("tempo_entrega_dias").cast(IntegerType()),
        col("tempo_entrega_estimado_dias").cast(IntegerType()),
        col("cidade").cast("STRING"),
        col("estado").cast("STRING")
    )
    .dropDuplicates(["id_pedido"])
)

# Salvar na Camada Gold
target_table = "gold.ft_atrasos_pedidos_local_vendedor"
(df_gold_atrasos.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable(target_table))

print(f"✅ Fato Gold '{target_table}' criada com sucesso.")

✅ Fato Gold 'gold.ft_atrasos_pedidos_local_vendedor' criada com sucesso.


#### Criação da View gold.view_tempo_medio_entrega_localidade

In [0]:
spark.sql("""
CREATE OR REPLACE VIEW gold.view_tempo_medio_entrega_localidade
AS
SELECT
    cidade, 
    estado, 
    ROUND(AVG(tempo_entrega_dias), 2) AS tempo_medio_entrega,           
    ROUND(AVG(tempo_entrega_estimado_dias), 2) AS tempo_medio_estimado, 
    CASE
        WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) THEN 'SIM'
        ELSE 'NÃO'
    END AS entrega_maior_que_estimado
FROM gold.ft_atrasos_pedidos_local_vendedor
WHERE entrega_no_prazo != 'Não Entregue' 
GROUP BY cidade, estado
ORDER BY tempo_medio_entrega DESC
""")

print("✅ View 'gold.view_tempo_medio_entrega_localidade' criada com sucesso.")

✅ View 'gold.view_tempo_medio_entrega_localidade' criada com sucesso.


#### Criação da View gold.view_vendedor_pontualidade

In [0]:
spark.sql("""
CREATE OR REPLACE VIEW gold.view_vendedor_pontualidade
AS
SELECT
    id_vendedor, 
    COUNT(id_pedido) AS total_pedidos, 
    SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) AS total_atrasados, 
    ROUND(
        (SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) * 100.0) / COUNT(id_pedido), 2
    ) AS percentual_atraso 
FROM gold.ft_atrasos_pedidos_local_vendedor
WHERE entrega_no_prazo != 'Não Entregue' 
GROUP BY id_vendedor
ORDER BY percentual_atraso DESC, total_pedidos DESC
""")

print("✅ View 'gold.view_vendedor_pontualidade' criada com sucesso.")

✅ View 'gold.view_vendedor_pontualidade' criada com sucesso.


# 3º Projeto: Área Comercial (Análises Temporais e de Vendas)

#### Criação da Dimensão de Tempo gold.dm_tempo

In [0]:
# Definir o range de datas
date_range_df = spark.sql("""
    SELECT 
        MIN(TO_DATE(pedido_compra_timestamp)) AS start_date,
        MAX(TO_DATE(pedido_compra_timestamp)) AS end_date
    FROM silver.ft_pedidos
""")
start_date = date_range_df.collect()[0]['start_date']
end_date = date_range_df.collect()[0]['end_date']

# Geração da sequência de datas
df_sequence = spark.sql(f"""
    SELECT 
        EXPLODE(SEQUENCE(
            TO_DATE('{start_date}', 'yyyy-MM-dd'), 
            TO_DATE('{end_date}', 'yyyy-MM-dd'),   
            INTERVAL '1' DAY
        )) AS sk_tempo
""")

# Enriquecimento da Dimensão
df_dim_tempo = (df_sequence
    .withColumn("ano", year(col("sk_tempo")).cast("INT"))
    .withColumn("trimestre", quarter(col("sk_tempo")).cast("INT"))
    .withColumn("mes", month(col("sk_tempo")).cast("INT"))
    .withColumn("semana_do_ano", dayofyear(col("sk_tempo")).cast("INT")) 
    .withColumn("dia", dayofmonth(col("sk_tempo")).cast("INT"))
    .withColumn("dia_da_semana_num", dayofweek(col("sk_tempo")).cast("INT"))
    .withColumn("dia_da_semana_nome", 
        when(dayofweek("sk_tempo") == 1, 'Domingo')
        .when(dayofweek("sk_tempo") == 2, 'Segunda')
        .when(dayofweek("sk_tempo") == 3, 'Terça')
        .when(dayofweek("sk_tempo") == 4, 'Quarta')
        .when(dayofweek("sk_tempo") == 5, 'Quinta')
        .when(dayofweek("sk_tempo") == 6, 'Sexta')
        .otherwise('Sábado')
    )
    .withColumn("mes_nome", date_format("sk_tempo", "MMMM"))
    .withColumn("eh_fim_de_semana", 
        when((dayofweek("sk_tempo") == 1) | (dayofweek("sk_tempo") == 7), "Sim")
        .otherwise("Não")
    )
    .select("sk_tempo", "ano", "trimestre", "mes", "semana_do_ano", "dia", 
            "dia_da_semana_num", "dia_da_semana_nome", "mes_nome", "eh_fim_de_semana")
)

# Salvar na Camada Gold
target_table = "gold.dm_tempo"
(df_dim_tempo.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable(target_table))

print(f"✅ Dimensão de Tempo '{target_table}' criada com sucesso no intervalo de {start_date} a {end_date}.")

✅ Dimensão de Tempo 'gold.dm_tempo' criada com sucesso no intervalo de 2016-09-04 a 2018-10-17.


#### Criação da Fato gold.ft_vendas_geral

In [0]:
# NOTA TÉCNICA AO AVALIADOR SOBRE COTAÇÃO DO DÓLAR (USD)
#
# PROBLEMA: O join entre a ft_vendas_geral e silver.dm_cotacao_dolar (que utiliza 
# dados simulados esparsos da Atividade 2) resultou em valores NULL para a 
# cotação na maioria dos registros de 2017 e 2018 (datas fora do range simulado).
# 
# SOLUÇÃO ADOTADA: Para garantir a integridade e a funcionalidade das Views 
# e KPIs (que dependem dos valores em USD), e para NÃO ALTERAR o notebook da 
# Camada Silver/Bronze já entregue, fiz o seguinte neste bloco:
#
#   1. Valor de Contingência (Fallback): Foi definido um valor padrão de 3.20 
#      para a cotação (VALOR_DOLAR_CONTINGENCIA).
#   2. Função COALESCE: Utilizei COALESCE(col("cotacao_dolar"), 3.20) para 
#      substituir o NULL pelo valor de contingência, garantindo que a divisão 
#      por zero ou por NULL não ocorra, e que os campos USD sejam populados.
#   3. Rastreabilidade: A coluna 'cotacao_dolar' no resultado final reflete o 
#      valor real usado na conversão (seja ele o simulado esparso, ou o 3.20).
#

# Carregar Fontes Silver
df_itens = spark.read.table("silver.ft_itens_pedidos")
df_pedidos = spark.read.table("silver.ft_pedidos")
df_cotacao = spark.read.table("silver.dm_cotacao_dolar")
df_avaliacoes = spark.read.table("silver.ft_avaliacoes_pedidos")

# Define o valor de contingência (3.20) para o dólar. 
VALOR_DOLAR_CONTINGENCIA = lit(3.20).cast(DecimalType(8, 4)) 

# Agregação de Avaliações
df_avaliacao_media = (df_avaliacoes
    .groupBy("id_pedido")
    .agg(round(avg(col("avaliacao")), 2).alias("avaliacao_pedido_media"))
)

# Join e Criação de Colunas Intermediárias
df_temp = (df_itens
    .join(df_pedidos.drop("ingestion_timestamp"), ["id_pedido"], "inner")
    .withColumn("data_pedido", to_date(col("pedido_compra_timestamp")).cast(DateType()))
    .join(df_cotacao.drop("ingestion_timestamp"), col("data_pedido") == col("data"), "left")
    .join(df_avaliacao_media, ["id_pedido"], "left")
    .withColumn("valor_total_item_brl_calc", (col("preco_BRL") + col("preco_frete")))
    .withColumn("cotacao_final", coalesce(col("cotacao_dolar"), VALOR_DOLAR_CONTINGENCIA))
)

# Projeção Final e Conversão USD
df_vendas = (df_temp
    .select(
        # Chaves e IDs
        col("id_pedido").cast("STRING"),
        col("id_item").cast("STRING"),
        col("id_consumidor").alias("fk_cliente").cast("STRING"),
        col("id_produto").alias("fk_produto").cast("STRING"),
        col("id_vendedor").alias("fk_vendedor").cast("STRING"),
        col("data_pedido").alias("fk_tempo").cast(DateType()), 
        
        # Logística e Avaliação
        col("status").alias("status_pedido").cast("STRING"),
        col("tempo_entrega_dias").cast("INT"),
        col("entrega_no_prazo").cast("STRING"),
        col("avaliacao_pedido_media").alias("avaliacao_pedido").cast(DecimalType(3, 2)),
        
        # Valores BRL
        col("preco_BRL").cast(DecimalType(12, 2)).alias("valor_produto_brl"),
        col("preco_frete").cast(DecimalType(12, 2)).alias("valor_frete_brl"),
        col("valor_total_item_brl_calc").cast(DecimalType(12, 2)).alias("valor_total_item_brl"),
        
        # Cotação usada (Reflete o valor que foi usado, seja ele real ou contingência)
        col("cotacao_final").alias("cotacao_dolar").cast(DecimalType(8, 4)), 
        
        # Conversão para USD (USANDO cotacao_final para a divisão)
        round(col("preco_BRL") / col("cotacao_final"), 2).cast(DecimalType(12, 2)).alias("valor_produto_usd"),
        round(col("preco_frete") / col("cotacao_final"), 2).cast(DecimalType(12, 2)).alias("valor_frete_usd"),
        round(col("valor_total_item_brl_calc") / col("cotacao_final"), 2).cast(DecimalType(12, 2)).alias("valor_total_item_usd")
    )
)

# Salvar na Camada Gold
target_table = "gold.ft_vendas_geral"
(df_vendas.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable(target_table))

print(f"✅ Fato Gold '{target_table}' criada com sucesso, usando valor de contingência (3.20) para cotações faltantes.")

✅ Fato Gold 'gold.ft_vendas_geral' criada com sucesso, usando valor de contingência (3.20) para cotações faltantes.


#### Criação da View gold.view_vendas_por_periodo

In [0]:
spark.sql("""
CREATE OR REPLACE VIEW gold.view_vendas_por_periodo
AS
SELECT
    T.ano,
    T.trimestre,
    T.mes,
    T.mes_nome,                 
    T.dia,
    T.dia_da_semana_num,
    T.dia_da_semana_nome,  
    
    COUNT(DISTINCT F.id_pedido) AS total_pedidos,
    COUNT(F.id_item) AS total_itens,
    ROUND(SUM(F.valor_total_item_brl), 2) AS receita_total_brl,
    ROUND(SUM(F.valor_total_item_usd), 2) AS receita_total_usd,
    
    ROUND(AVG(F.valor_total_item_brl), 2) AS ticket_medio_brl,
    
    ROUND(AVG(CASE WHEN F.status_pedido IN ('entregue', 'enviado') THEN F.avaliacao_pedido ELSE NULL END), 2) AS avaliacao_media
    
FROM gold.ft_vendas_geral F
INNER JOIN gold.dm_tempo T ON F.fk_tempo = T.sk_tempo
GROUP BY 
    T.ano, 
    T.trimestre, 
    T.mes, 
    T.mes_nome, 
    T.dia, 
    T.dia_da_semana_num,
    T.dia_da_semana_nome
ORDER BY T.ano, T.mes, T.dia
""")

print("✅ View 'gold.view_vendas_por_periodo' recriada com sucesso (Campos de nome corrigidos).")

✅ View 'gold.view_vendas_por_periodo' recriada com sucesso (Campos de nome corrigidos).


#### Consultas Analíticas (na View)

In [0]:
print("📈 Consulta 4.4.1: Dia da Semana com Maior Receita (BRL)")
spark.sql("""
SELECT
    dia_da_semana_nome,
    ROUND(SUM(receita_total_brl), 2) AS receita_total_agregada
FROM gold.view_vendas_por_periodo
GROUP BY dia_da_semana_nome
ORDER BY receita_total_agregada DESC
LIMIT 1
""").show()

print("📈 Consulta 4.4.2: Mês com Maior Ticket Médio no Último Ano")
spark.sql("""
SELECT
    mes_nome,
    mes,
    ROUND(AVG(ticket_medio_brl), 2) AS ticket_medio_mensal
FROM gold.view_vendas_por_periodo
WHERE ano = (SELECT MAX(ano) FROM gold.dm_tempo) 
GROUP BY mes_nome, mes
ORDER BY ticket_medio_mensal DESC
LIMIT 1
""").show()

📈 Consulta 4.4.1: Dia da Semana com Maior Receita (BRL)
+------------------+----------------------+
|dia_da_semana_nome|receita_total_agregada|
+------------------+----------------------+
|           Segunda|            2600533.82|
+------------------+----------------------+

📈 Consulta 4.4.2: Mês com Maior Ticket Médio no Último Ano
+---------+---+-------------------+
| mes_nome|mes|ticket_medio_mensal|
+---------+---+-------------------+
|September|  9|             166.46|
+---------+---+-------------------+



#### Criação da gold.view_top_produto

In [0]:
spark.sql("""
CREATE OR REPLACE VIEW gold.view_top_produto
AS
SELECT
    F.fk_produto AS id_produto,
    P.categoria_produto,
    
    COUNT(F.id_item) AS quantidade_vendida,
    COUNT(DISTINCT F.id_pedido) AS total_pedidos,
    
    ROUND(SUM(F.valor_total_item_brl), 2) AS receita_brl,
    ROUND(SUM(F.valor_total_item_usd), 2) AS receita_usd,
    
    ROUND(AVG(F.valor_produto_brl), 2) AS preco_medio_brl,
    
    ROUND(AVG(F.avaliacao_pedido), 2) AS avaliacao_media,
    
    ROUND(AVG(P.peso_produto_gramas), 2) AS peso_medio_gramas
    
FROM gold.ft_vendas_geral F
INNER JOIN silver.ft_produtos P ON F.fk_produto = P.id_produto
GROUP BY F.fk_produto, P.categoria_produto
ORDER BY receita_brl DESC
""")

print("✅ View 'gold.view_top_produto' criada com sucesso.")

✅ View 'gold.view_top_produto' criada com sucesso.


#### Criação da view_vendas_produtos_esteticos

In [0]:
sql_view_esteticos = """
CREATE OR REPLACE VIEW gold.view_vendas_produtos_esteticos
AS

WITH CategoriaFashion AS (
    -- CTE para filtrar e agregar os dados de itens de moda/estética por período
    SELECT
        T.ano,
        T.mes,
        P.categoria_produto,
        F.id_pedido,
        F.id_item,
        F.valor_total_item_brl,
        F.valor_total_item_usd,
        F.avaliacao_pedido,
        F.status_pedido
    FROM gold.ft_vendas_geral F
    INNER JOIN gold.dm_tempo T ON F.fk_tempo = T.sk_tempo
    INNER JOIN silver.ft_produtos P ON F.fk_produto = P.id_produto
    -- Requisito: Inclui apenas categorias que iniciam com "fashion"
    WHERE P.categoria_produto LIKE 'fashion%' 
)
SELECT
    ano,
    mes,
    categoria_produto,
    
    COUNT(DISTINCT id_pedido) AS total_pedidos,
    COUNT(id_item) AS total_itens_vendidos,
    
    ROUND(SUM(valor_total_item_brl), 2) AS receita_total_brl,
    ROUND(SUM(valor_total_item_usd), 2) AS receita_total_usd,
    
    ROUND(AVG(valor_total_item_brl), 2) AS ticket_medio_brl,
    ROUND(AVG(valor_total_item_usd), 2) AS ticket_medio_usd,
    
    -- Média das avaliações dos pedidos entregues
    ROUND(AVG(CASE WHEN status_pedido = 'entregue' THEN avaliacao_pedido ELSE NULL END), 2) AS avaliacao_media 
    
FROM CategoriaFashion
GROUP BY ano, mes, categoria_produto
ORDER BY ano, mes, receita_total_brl DESC
"""

spark.sql(sql_view_esteticos)
print("✅ View 'gold.view_vendas_produtos_esteticos' (usando CTE) criada com sucesso.")

✅ View 'gold.view_vendas_produtos_esteticos' (usando CTE) criada com sucesso.


#### Verificação final da tabela FATO CENTRAL

In [0]:
print("✅ Verificação Estrutural da gold.ft_vendas_geral (Conferência com Exemplo 3.2):")

spark.read.table("gold.ft_vendas_geral").printSchema()

spark.read.table("gold.ft_vendas_geral").limit(5).show(truncate=False)

✅ Verificação Estrutural da gold.ft_vendas_geral (Conferência com Exemplo 3.2):
root
 |-- id_pedido: string (nullable = true)
 |-- id_item: string (nullable = true)
 |-- fk_cliente: string (nullable = true)
 |-- fk_produto: string (nullable = true)
 |-- fk_vendedor: string (nullable = true)
 |-- fk_tempo: date (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- tempo_entrega_dias: integer (nullable = true)
 |-- entrega_no_prazo: string (nullable = true)
 |-- valor_produto_brl: decimal(12,2) (nullable = true)
 |-- valor_frete_brl: decimal(12,2) (nullable = true)
 |-- valor_total_item_brl: decimal(12,2) (nullable = true)
 |-- cotacao_dolar: decimal(8,4) (nullable = true)
 |-- valor_produto_usd: decimal(12,2) (nullable = true)
 |-- valor_frete_usd: decimal(12,2) (nullable = true)
 |-- valor_total_item_usd: decimal(12,2) (nullable = true)
 |-- avaliacao_pedido: decimal(3,2) (nullable = true)

+--------------------------------+-------+--------------------------------+-------